In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
# KNN Image Classifier - Cat vs Dog vs Panda

import wandb
from sklearn.model_selection import train_test_split
import os, cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch, torchvision
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# WANDB setup
wandb.init(project="Dog-or-Cat-or-Panda")

# Device
device = 'cpu'

# Constants
IMAGE_SIZE = (64, 64)
DATA_DIR = "CatDogPanda"
LABELS = ["cat", "dog", "panda"]

# Load images and labels
def load_images(data_dir):
    X = []
    y = []
    for label in LABELS:
        path = os.path.join(data_dir, label)
        for file in tqdm(os.listdir(path), desc=f"Loading {label} images"):
            img_path = os.path.join(path, file)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, IMAGE_SIZE)
                X.append(img)
                y.append(label)
    return np.array(X), np.array(y)

X, y = load_images(DATA_DIR)

# Normalize
X = X / 255.0

# Flatten the image for KNN input
X_flat = X.reshape((X.shape[0], -1))

# Encode labels to integers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_flat, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Train KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Predict
y_pred = knn.predict(X_test)

# Evaluation
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=LABELS))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Log results to wandb
wandb.log({
    "accuracy": (y_pred == y_test).sum() / len(y_test),
    "confusion_matrix": wandb.sklearn.plot_confusion_matrix(y_test, y_pred, labels=LABELS),
    "classification_report": classification_report(y_test, y_pred, output_dict=True, target_names=LABELS)
})


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter: